## Import Libraries

In [3]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist,jaccard

## Load Data

#### Load DrugBank Data

In [4]:
df = pd.read_csv('Input/drugbank_targets.csv')

In [5]:
df.head()

,DrugBank ID,Name,Type,UniProt ID,UniProt Name
0,DB00001,Lepirudin,BiotechDrug,P00734,Prothrombin
1,DB00002,Cetuximab,BiotechDrug,P00533,Epidermal growth factor receptor
2,DB00002,Cetuximab,BiotechDrug,O75015,Low affinity immunoglobulin gamma Fc region re...
3,DB00002,Cetuximab,BiotechDrug,P00736,Complement C1r subcomponent
4,DB00002,Cetuximab,BiotechDrug,P02745,Complement C1q subcomponent subunit A


In [6]:
df.shape

(18655, 5)

In [7]:
df['Type'].unique()

array(['BiotechDrug', 'SmallMoleculeDrug'], dtype=object)

In [8]:
len(df['Name'].unique())

7137

In [9]:
#df['UniProtID'] = df['UniProt ID']

#### Load UniProt ID table

In [10]:
uniprot = pd.read_table('Input/gene_to_uniprot.txt')

In [11]:
uniprot.head()

,Approved Symbol,UniProt ID
0,A1BG,P04217
1,A1BG-AS1,NaN
2,A1CF,Q9NQ94
3,A2M,P01023
4,A2M-AS1,NaN


In [12]:
uniprot.shape

(41375, 2)

In [13]:
uniprot = uniprot[pd.notnull(uniprot['UniProt ID'])]

## Map UniProt ID to Gene

In [14]:
df.set_index('UniProt ID', inplace = True)
uniprot.set_index('UniProt ID',inplace=True)

In [15]:
uniprot.head()

,Approved Symbol
UniProt ID,
P04217,A1BG
Q9NQ94,A1CF
P01023,A2M
A8K2U0,A2ML1
U3KPV4,A3GALT2


In [16]:
df = pd.merge(df, uniprot, how= 'left', on = 'UniProt ID')

## Set up Matrix

In [48]:
df = df.rename(index=str, columns = {'Approved Symbol':'Targets'})

In [49]:
df.head()

,index,UniProt ID,DrugBank ID,Name,Type,UniProt Name
Targets,,,,,,
F2,0,P00734,DB00001,Lepirudin,BiotechDrug,Prothrombin
EGFR,1,P00533,DB00002,Cetuximab,BiotechDrug,Epidermal growth factor receptor
FCGR3B,2,O75015,DB00002,Cetuximab,BiotechDrug,Low affinity immunoglobulin gamma Fc region re...
C1R,3,P00736,DB00002,Cetuximab,BiotechDrug,Complement C1r subcomponent
C1QA,4,P02745,DB00002,Cetuximab,BiotechDrug,Complement C1q subcomponent subunit A


In [30]:
df.reset_index(inplace=True)
df.set_index('Targets', inplace=True)

In [28]:
targets = df['Targets'].drop_duplicates()

In [51]:
targets_and_names = df['Name']

In [25]:
# sim_df = 1 - pdist(df[['Targets', 'Name']],'jaccard')
# sim_matrix = pd.DataFrame(squareform(sim_df), index = df.index, columns = df.index)

ValueError: could not convert string to float: 'Tildrakizumab'

In [61]:
df.iloc[:,[3]]

,Name
Targets,
F2,Lepirudin
EGFR,Cetuximab
FCGR3B,Cetuximab
C1R,Cetuximab
C1QA,Cetuximab
C1QB,Cetuximab
C1QC,Cetuximab
FCGR3A,Cetuximab
C1S,Cetuximab


In [70]:
pd.get_dummies(df.iloc[:,[3]])

In [74]:
df.to_csv('ok.csv')